# BIS SDMX RESTful API

Downloading central bank rates uing RESTful API from BIS Stats page

Spec at: https://stats.bis.org/api-doc/v1/

In [1]:
import requests
import pandas as pd
import xmltodict
from datetime import date

from SqlAlquemyInsertMarketDataHandler import SqlAlquemyInsertMarketDataHandler

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
start_date = '2023-09-01'
end_date = str(date.today())

url = f'https://stats.bis.org/api/v1/data/BIS%2CWS_CBPOL_D%2C1.0/all/all?startPeriod={start_date}&detail=dataonly'
response = requests.get(url)
python_dict = xmltodict.parse(response.text)
dataset = python_dict['message:StructureSpecificData']['message:DataSet']['Series']

In [3]:
df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq='B'))

for i in range(len(dataset)):
    country_code = dataset[i]['@REF_AREA']
    df_parsed = pd.DataFrame(dataset[i]['Obs'])
    df_parsed.index = pd.to_datetime(df_parsed['@TIME_PERIOD'])
    if country_code == 'XM':
        euro_zone_countries = ['AT','BE','DE','ES','FI','FR','GR','IE','IT','NL','PT']
        for eu_country in euro_zone_countries:
            df[eu_country + '_IR'] = df_parsed['@OBS_VALUE'].astype(float)
    else:
        df[country_code + '_IR'] = df_parsed['@OBS_VALUE'].astype(float)

df = df.ffill()
df = df.bfill()

In [4]:
sql_handler = SqlAlquemyInsertMarketDataHandler()
symbol_codes = sql_handler.get_symbol_codes(source='BIS')
df_filtered = df[symbol_codes]
df_filtered

,AR_IR,AT_IR,AU_IR,BE_IR,BR_IR,CA_IR,CH_IR,CL_IR,CN_IR,CO_IR,...,PH_IR,PL_IR,PT_IR,RU_IR,SA_IR,SE_IR,TH_IR,TR_IR,US_IR,ZA_IR
2023-09-01,118.0,4.25,4.10,4.25,13.25,5.0,1.75,10.25,3.45,13.25,...,6.25,6.75,4.25,12.0,6.0,3.75,2.25,25.0,5.375,8.25
2023-09-04,118.0,4.25,4.10,4.25,13.25,5.0,1.75,10.25,3.45,13.25,...,6.25,6.75,4.25,12.0,6.0,3.75,2.25,25.0,5.375,8.25
2023-09-05,118.0,4.25,4.10,4.25,13.25,5.0,1.75,10.25,3.45,13.25,...,6.25,6.75,4.25,12.0,6.0,3.75,2.25,25.0,5.375,8.25
2023-09-06,118.0,4.25,4.10,4.25,13.25,5.0,1.75,9.50,3.45,13.25,...,6.25,6.75,4.25,12.0,6.0,3.75,2.25,25.0,5.375,8.25
2023-09-07,118.0,4.25,4.10,4.25,13.25,5.0,1.75,9.50,3.45,13.25,...,6.25,6.00,4.25,12.0,6.0,3.75,2.25,25.0,5.375,8.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01,133.0,4.50,4.35,4.50,11.75,5.0,1.75,8.25,3.45,13.00,...,6.50,5.75,4.50,16.0,6.0,4.00,2.50,42.5,5.375,8.25
2024-01-02,133.0,4.50,4.35,4.50,11.75,5.0,1.75,8.25,3.45,13.00,...,6.50,5.75,4.50,16.0,6.0,4.00,2.50,42.5,5.375,8.25
2024-01-03,133.0,4.50,4.35,4.50,11.75,5.0,1.75,8.25,3.45,13.00,...,6.50,5.75,4.50,16.0,6.0,4.00,2.50,42.5,5.375,8.25
2024-01-04,133.0,4.50,4.35,4.50,11.75,5.0,1.75,8.25,3.45,13.00,...,6.50,5.75,4.50,16.0,6.0,4.00,2.50,42.5,5.375,8.25


In [5]:
save_start_date = '2023-10-01'
df_rates_changes = df_filtered[df_filtered.diff() != 0]
rates_to_save = df_rates_changes[save_start_date:]

for symbol_code in rates_to_save:
    records_to_save = rates_to_save[symbol_code].dropna()
    if len(records_to_save) > 0:
        sql_handler.delete_all_records(symbol_code, save_start_date)
        sql_handler.insert_into_table(symbol_code, records_to_save)
        inserted_count = sql_handler.get_indicator_count(
            symbol_code, save_start_date)
        print(f'{symbol_code}: insterted {inserted_count} rates')
    else: 
        print(f'{symbol_code}: no new rates since {save_start_date}')

AR_IR: insterted 1 rates
AT_IR: no new rates since 2023-10-01
AU_IR: insterted 1 rates
BE_IR: no new rates since 2023-10-01
BR_IR: insterted 2 rates
CA_IR: no new rates since 2023-10-01
CH_IR: no new rates since 2023-10-01
CL_IR: insterted 2 rates
CN_IR: no new rates since 2023-10-01
CO_IR: insterted 1 rates
CZ_IR: insterted 1 rates
DE_IR: no new rates since 2023-10-01
DK_IR: no new rates since 2023-10-01
ES_IR: no new rates since 2023-10-01
FI_IR: no new rates since 2023-10-01
FR_IR: no new rates since 2023-10-01
GB_IR: no new rates since 2023-10-01
GR_IR: no new rates since 2023-10-01
HK_IR: no new rates since 2023-10-01
HU_IR: insterted 3 rates
ID_IR: insterted 1 rates
IE_IR: no new rates since 2023-10-01
IL_IR: no new rates since 2023-10-01
IN_IR: no new rates since 2023-10-01
IT_IR: no new rates since 2023-10-01
JP_IR: no new rates since 2023-10-01
KR_IR: no new rates since 2023-10-01
MX_IR: no new rates since 2023-10-01
MY_IR: no new rates since 2023-10-01
NL_IR: no new rates sin